## part 1 ##

In [1]:
ex = '''class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12'''.strip().split('\n')

In [2]:
def parse(lines):
    rangelines = lines[0:lines.index('')]
    ticket = lines[lines.index('your ticket:')+1]
    nearbytix = lines[lines.index('nearby tickets:')+1:]
    return rangelines, ticket, nearbytix

In [3]:
exrangelines, exticket, exnearbytix = parse(ex)
exrangelines, exticket, exnearbytix

(['class: 1-3 or 5-7', 'row: 6-11 or 33-44', 'seat: 13-40 or 45-50'],
 '7,1,14',
 ['7,3,47', '40,4,50', '55,2,20', '38,6,12'])

In [4]:
def get_limits(lines):
    limits = {}
    for line in lines:
        lhs, rhs = line.split(':')
        key = lhs.strip()
        lims = []
        for s in rhs.strip().split('or'):
            lo, hi = s.strip().split('-')
            lims.append((int(lo), int(hi)))
        limits[key] = lims
    return limits

In [5]:
exlims = get_limits(exrangelines)
exlims

{'class': [(1, 3), (5, 7)],
 'row': [(6, 11), (33, 44)],
 'seat': [(13, 40), (45, 50)]}

In [6]:
def invalidnums(ticket, limits):
    invalid = []
    for num in ticket:
        checks = []
        for key in limits:
            for r in limits[key]:
                checks.append(r[0] <= num <= r[1])
        if not any(checks):
            invalid.append(num)
    return invalid

In [7]:
invalidnums([7,3,47], exlims)

[]

In [8]:
invalidnums([40, 4, 50], exlims)

[4]

In [9]:
exnearbytix

['7,3,47', '40,4,50', '55,2,20', '38,6,12']

In [10]:
def process_nearbytix(nearbytix, limits):
    badnums = []
    for ticket in nearbytix:
        vals = [int(num) for num in ticket.split(',')]
        invalid = invalidnums(vals, limits)
        if invalid:
            badnums.extend(invalid)
    return sum(badnums)

In [11]:
process_nearbytix(exnearbytix, exlims)

71

In [12]:
with open('inputs/day16.input') as fp:
    data  = fp.read().split('\n')
dataranges, dataticket, datanearbytix = parse(data)
dataticket = [int(x) for x in ]

In [13]:
datalims = get_limits(dataranges)

In [14]:
process_nearbytix(datanearbytix[:-1], datalims)

32842

## part 2 ##

In [15]:
def good_nearbytix(nearbytix, limits):
    good = []
    for ticket in nearbytix:
        vals = [int(num) for num in ticket.split(',')]
        invalid = invalidnums(vals, limits)
        if not invalid:
            good.append([int(t) for t in ticket.split(',')])
    return good

In [16]:
ex2 = '''class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9'''.strip().split('\n')

In [17]:
ex2rangelines, ex2ticket, ex2nearbytix = parse(ex2)
ex2rangelines, ex2ticket, ex2nearbytix

(['class: 0-1 or 4-19', 'row: 0-5 or 8-19', 'seat: 0-13 or 16-19'],
 '11,12,13',
 ['3,9,18', '15,1,5', '5,14,9'])

In [18]:
ex2lims = get_limits(ex2rangelines)
ex2lims

{'class': [(0, 1), (4, 19)],
 'row': [(0, 5), (8, 19)],
 'seat': [(0, 13), (16, 19)]}

In [19]:
ex2good = good_nearbytix(ex2nearbytix, ex2lims)
ex2good

[[3, 9, 18], [15, 1, 5], [5, 14, 9]]

In [20]:
import collections

In [21]:
def is_num_valid_in_field(num, field_ranges):
    return any((r[0] <= num <= r[1]) for r in field_ranges)

In [22]:
is_num_valid_in_field(3, ex2lims['seat'])

True

In [23]:
def find_possible(goodtix, lims):
    num_fields = len(goodtix[0])
    valid = collections.defaultdict(list)
    for i in range(num_fields):
        nums = [int(ticket[i]) for ticket in goodtix]
        for key in lims:
            if all(is_num_valid_in_field(num, lims[key]) for num in nums):
                valid[i].append(key)
    return valid
        

In [24]:
ex2possible = find_possible(ex2good, ex2lims)
ex2possible

defaultdict(list,
            {0: ['row'], 1: ['class', 'row'], 2: ['class', 'row', 'seat']})

In [25]:
def resolve_fields(possible, assigned={}):
    if possible == {}:
        return assigned
    found = {}
    for key in possible:
        if len(possible[key]) == 1:
            found[key] = possible[key][0]
    for key in found:
        del possible[key]
    for key in possible:
        possible[key] = [v for v in possible[key] if v not in found.values()]
    assigned.update(found)
    return resolve_fields(possible, assigned)


In [26]:
ex2resolved = resolve_fields(ex2possible.copy())
ex2resolved

{0: 'row', 1: 'class', 2: 'seat'}

In [27]:
datagood = good_nearbytix(datanearbytix[:-1], datalims)

In [28]:
datapossible = find_possible(datagood, datalims)

In [29]:
dataresolved = resolve_fields(datapossible.copy())
dataresolved

{0: 'arrival platform',
 1: 'class',
 2: 'arrival location',
 11: 'type',
 5: 'price',
 8: 'departure location',
 18: 'departure platform',
 17: 'departure date',
 9: 'departure track',
 4: 'departure station',
 7: 'departure time',
 16: 'duration',
 19: 'wagon',
 12: 'route',
 13: 'arrival track',
 3: 'seat',
 15: 'train',
 6: 'row',
 10: 'zone',
 14: 'arrival station'}

In [30]:
prod = 1
for key in dataresolved:
    if 'departure' in dataresolved[key]:
        prod *= int(dataticket.split(',')[key])
print(prod)

2628667251989
